# Import

In [ ]:
import os
import numpy as np
import pandas as pd
import spacy

import matplotlib.pyplot as plt
import seaborn as sns

sns.set
%matplotlib inline

# Load the dataset

In [ ]:
path = '../input'
df_train = pd.read_csv(os.path.join(path, 'train.csv'))
df_test = pd.read_csv(os.path.join(path, 'test.csv'))

# Look at the dataset

In [ ]:
df_train.head()

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train.info()

The data is clean, there is no Naan values

In [ ]:
df_train['target'].value_counts().plot(kind='bar');

In [ ]:
insincere_ratio = (80810 / 1225312) * 100
insincere_ratio

In [ ]:
y = df_train['target']
X = df_train['question_text']

In [ ]:
X_insincere = X[y == 1]
X_insincere.head()

We can already notice the troll content within the questions.

In [ ]:
X_sincere = X[y == 0]
X_sincere.head()

Whereas within the sincere question, the questions are legit.

# Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Tokenizing

Creation of a tokenize's function permitting to automatically tokenize our train and test set

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
def tokenize(data):
    corpus = [word_tokenize(token) for token in data]
    lowercase_train = [[token.lower() for token in doc] for doc in corpus]
    alphas = [[token for token in doc if token.isalpha()] for doc in lowercase_train]
    stop_words = stopwords.words('english')
    train_no_stop = [[token for token in doc if token not in stop_words] for doc in alphas]
    stemmer = PorterStemmer()
    stemmed = [[stemmer.stem(token) for token in doc] for doc in train_no_stop]
    train_clean_str = [ ' '.join(doc) for doc in stemmed]
    return train_clean_str

In [ ]:
X_train = tokenize(X_train)
X_test = tokenize(X_test)

# Machine Learning

## Preprocessing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
tvec = TfidfVectorizer(stop_words='english')
cvec = CountVectorizer(stop_words='english')
svd = TruncatedSVD(n_components=100, random_state=42)

## Machine learning models

### MultinomialNB

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb = MultinomialNB()

In [ ]:
pipe = Pipeline([('vectorizer', cvec), ('mnb', mnb)])

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
y_pred = pipe.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
labels = ['sincere', 'unsincere']
cm = pd.DataFrame(cm, columns=labels, index=labels)
cm

## Validation

In [ ]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(pipe, X_test, y_test, scoring='f1_macro', cv=5).mean()

In [ ]:
cv

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred)
print(cr)

# Test

In [ ]:
test = df_test['question_text']
test = tokenize(test)

## Machine Learning

In [ ]:
y_pred = pipe.predict(test)
y_pred

# Submit on the sample submission

In [ ]:
path = '../input'
df_sub = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [ ]:
df_sub['prediction'] = y_pred

In [ ]:
df_sub.to_csv("submission.csv", index=False)